## 축제 기간 OD 데이터 분석

#### 원본 데이터 병합 및 행정동 변환

In [ ]:
import os
import pandas as pd

def merge_csv(folder):
    csv_files = []
    for file_name in os.listdir(folder):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder, file_name)
            df = pd.read_csv(file_path)
            csv_files.append(df)
    merged = pd.concat(csv_files, ignore_index=True)
    return merged


In [ ]:
early_sep = merge_csv('od_20230901_10')
mid_sep = merge_csv('od_20230911_20')
end_sep = merge_csv('od_20230921_30')
early_oct = merge_csv('od_20231001_15')

In [ ]:
#행정동 변환하기
hdong = pd.read_csv('데이터분석 분야_데이터정의서\KIKmix_20230701.csv')

hdong = hdong.groupby(['행정동코드','시도명','시군구명','읍면동명'], as_index=False).first()
hdong['주소'] = (hdong['시도명'] + ' ' + hdong['시군구명'] + ' ' + hdong['읍면동명']).fillna('').str.strip()

In [ ]:
def get_hdong(file):
    file = file.merge(hdong[['행정동코드','주소']], how='left', left_on='origin_hdong_cd', right_on='행정동코드')
    file['origin_hdong_cd'] = file['주소'].fillna(file['origin_hdong_cd'])
    file.drop(columns=['행정동코드','주소'], inplace=True)

    file = file.merge(hdong[['행정동코드','주소']], how='left', left_on='dest_hdong_cd', right_on='행정동코드')
    file['dest_hdong_cd'] = file['주소'].fillna(file['dest_hdong_cd'])
    file.drop(columns=['행정동코드','주소'], inplace=True)

    return file

In [ ]:
sep1 = get_hdong(early_sep)
sep2 = get_hdong(mid_sep)
sep3 = get_hdong(end_sep)

### 0. 데이터 필터링
| 축제 이름             | 비축제 기간         | 축제 기간       |분석 지역          |
|----------------------|--------------------|----------------|-------------------|
| 소래포구 축제         | 09/01 - 09/14     | 09/15 - 09/17 | 축제 개최 행정동 (논현1동,논현2동)|
| 진주남강유등축제      | 09/23 - 10/07     | 10/08 - 10/15 | 임시 주차장 및 축제지 소재 법정동 (경상남도 진주시 이현동, 초장동 외 5개)|
- 외지인은 출발지가 축제 개최지 (시 기준) 외의 지역인 사람으로 정의

In [20]:
import pandas as pd
import plotly.express as px

In [ ]:
sep1 = pd.read_csv('sep_hdong.csv')
sep2 = pd.read_csv('mid_hdong.csv')
sep3 = pd.read_csv('end_hdong.csv')
oct1 = pd.read_csv('oct_hdong.csv')

- 날짜, 지역 조건으로 데이터 필터링

In [66]:
# 진주 남강 유등축제 데이터 추출
parks = ['경상남도 진주시 이현동', '경상남도 진주시 초장동', '경상남도 진주시 충무공동', '경상남도 진주시 가호동', '경상남도 진주시 평거동', '경상남도 진주시 천전동', '경상남도 진주시 성북동']
prev1 = sep3[(sep3['date'] >= 20230923) & (sep3['dest_hdong_cd'].isin(parks) | sep3['origin_hdong_cd'].isin(parks))]
prev2 = oct1[(oct1['dest_hdong_cd'].isin(parks))|oct1['origin_hdong_cd'].isin(parks)] 

jinju = pd.concat([prev1, prev2])

# 인천 소래포구 축제 데이터 추출
ipark = ['인천광역시 남동구 논현1동', '인천광역시 남동구 논현2동']
prev = sep1[sep1['dest_hdong_cd'].isin(ipark)]
incheon = sep2[(sep2['date']<=20230917) & (sep2['dest_hdong_cd'].isin(ipark)|sep2['origin_hdong_cd'].isin(ipark))]

incheon = pd.concat([prev, incheon])

In [67]:
# 축제 기간 / 비축제 기간 구분
incheon['축제'] = '소래포구 축제'
jinju['축제'] = '진주남강유등 축제'

incheon['기간'] = incheon['date'].apply(lambda x : '축제' if (x >= 20230915) & (x <= 20230917) else '비축제')
jinju['기간'] = jinju['date'].apply(lambda x : '축제' if (x >= 20231008) & (x <= 20231015) else '비축제')

- 기타 필터링 조건 함수

In [101]:
#1. 현지인/외지인 표기
def local(data, region):
    data['구분'] = data['origin_hdong_cd'].apply(lambda x: '현지인' if region in x else '외지인')
    return data

#2. 쇼핑/여가/여행 목적 방문 필터링
def purpose_leisure (data):
    return data[(data['dest_purpose']==3) | (data['dest_purpose']==5)] 

#3. 축제지 인근 통행량 파악을 위한 데이터 필터링
def time_moved (data, parks) :
    data['time'] = data.apply(
        lambda row: row['start_time'] if row['origin_hdong_cd'] in parks else 
                    row['end_time'] if row['dest_hdong_cd'] in parks else None,
        axis=1
    )
    return data

#4. 목적지 열 기준 추가 필터링
def dest_filter(data, dest):
    return data[(data['dest_hdong_cd'].isin(dest))]

### 비축제 기간 대비 축제 기간 방문객 수
`분석 목표` :  현지인 통행량과 외지인 통행량을 나눠서 분석 (비축제기간 vs 축제기간 일별 평균 방문자 수)

#### 필터링 조건 적용

In [80]:
visitor_jinju = dest_filter(jinju, parks) ;visitor_incheon = dest_filter(incheon, ipark) # 목적지가 축제지 개최 행정동인 경우만 사용
visitor_jinju = purpose_leisure(visitor_jinju) ; visitor_incheon = purpose_leisure(visitor_incheon) # 쇼핑/여가/여행 목적 방문 필터링

visitor_jinju = local(visitor_jinju, '경상남도 진주') ; visitor_incheon = local(visitor_incheon, '인천광역시') # 현지인/외지인 구분

df_visitor = pd.concat([visitor_incheon, visitor_jinju])

# 축제별, 현지인/외지인별 통행량 집계
grouped_df = df_visitor.groupby(['축제', '구분','기간'])['od_cnts'].sum().reset_index()

In [82]:
# 일별 방문객 평균 계산
def average(x):
    if x['축제'] == '소래포구 축제':
        if x['기간'] =='비축제':
            return x['od_cnts']/14
        else:
            return x['od_cnts']/3
    else:
        if x['기간'] =='비축제':
            return x['od_cnts']/14
        else:
            return x['od_cnts']/8
        
grouped_df['avg'] = grouped_df.apply(lambda row: average(row), axis=1)

In [83]:
grouped_df

,축제,구분,기간,od_cnts,avg
0,소래포구 축제,외지인,비축제,19694,1406.714286
1,소래포구 축제,외지인,축제,6960,2320.000000
2,소래포구 축제,현지인,비축제,54457,3889.785714
3,소래포구 축제,현지인,축제,17269,5756.333333
4,진주남강유등 축제,외지인,비축제,9710,693.571429
5,진주남강유등 축제,외지인,축제,8825,1103.125000
6,진주남강유등 축제,현지인,비축제,38890,2777.857143
7,진주남강유등 축제,현지인,축제,25560,3195.000000


#### 시각화 결과

In [84]:
# 색상을 구분(현지인/외지인)에 따라 지정
color_map = {
    '비축제': '#4BA07E', 
    '축제': '#0e6d62'  
}

# Plotly를 이용한 시각화 (막대 그래프)
fig = px.bar(grouped_df, x='구분', y='avg', color='기간', barmode='group',
             facet_col='축제',  # 축제별로 그래프를 나눔
             color_discrete_map=color_map,  # 색상 맵핑 추가
             labels={'avg': '평균 통행량', '구분': '방문자 유형', '기간': '기간'},
             title='현지인/외지인별 축제기간 및 비축제기간 평균 통행량 비교')

# 그래프 출력
fig.show()

### 축제 기간 목적별 차량 이용률
`분석 목표` : 비축제기간 대비 축제기간 목적별 차량 이용률 추세 파악

#### 필터링 조건 적용 및 연산

In [95]:
modal_jinju = local(jinju, '경상남도 진주') ; modal_incheon = local(incheon, '인천광역시') # 현지인/외지인 구분
df_modal = pd.concat([modal_incheon, modal_jinju])

df_vehicle = df_modal[df_modal['modal'] == 0] # 차량 이용 데이터만 필터링 (modal == 0)

vehicle_count = df_vehicle.groupby(['축제','기간', 'dest_purpose']).size().reset_index(name='vehicle_count') # 각 기간과 목적별 차량 이용 건수 계산
total_count = df_modal.groupby(['축제','기간','dest_purpose']).size().reset_index(name='total_count') # 기간별 전체 통행 건수

df_vehicle = pd.merge(vehicle_count, total_count, on=['축제', '기간','dest_purpose'], how='left') # 차량 이용 건수와 전체 통행 건수를 병합
df_vehicle['modal_ratio'] = df_vehicle['vehicle_count'] / df_vehicle['total_count'] # 차량 이용 비율 계산

purposes = {0: '귀가', 1 : '업무', 2: '학업', 3: '쇼핑/여가', 4: '기타', 5:'여행'} # 목적별 코드를 목적명으로 변환
df_vehicle['dest_purpose'] = df_vehicle['dest_purpose'].map(purposes)

df_vehicle.head()

,축제,기간,dest_purpose,vehicle_count,total_count,modal_ratio
0,소래포구 축제,비축제,귀가,44999,58540,0.768688
1,소래포구 축제,비축제,업무,13029,17940,0.726254
2,소래포구 축제,비축제,학업,970,1339,0.724421
3,소래포구 축제,비축제,쇼핑/여가,6304,7667,0.822225
4,소래포구 축제,비축제,기타,15901,19178,0.829127


#### 시각화 결과

In [98]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# 축제별 데이터 분리
df_jinju = df_vehicle[df_vehicle['축제'] == '진주남강유등 축제']
df_incheon = df_vehicle[df_vehicle['축제'] == '소래포구 축제']

# 서브플롯 생성 (2개의 열)
fig = make_subplots(rows=1, cols=2, subplot_titles=("진주남강유등 축제", "소래포구 축제"))

# 왼쪽 서브플롯: 진주남강유등축제
for purpose in df_jinju['dest_purpose'].unique():
    df_subset = df_jinju[df_jinju['dest_purpose'] == purpose]
    fig.add_trace(
        go.Scatter(x=df_subset['기간'], y=df_subset['modal_ratio'], mode='lines+markers', name=purpose),
        row=1, col=1
    )

# 오른쪽 서브플롯: 소래포구 축제
for purpose in df_incheon['dest_purpose'].unique():
    df_subset = df_incheon[df_incheon['dest_purpose'] == purpose]
    fig.add_trace(
        go.Scatter(x=df_subset['기간'], y=df_subset['modal_ratio'], mode='lines+markers', name=purpose, showlegend=False),
        row=1, col=2
    )

# 레이아웃 설정
fig.update_layout(
    title_text="축제기간과 비축제기간 목적별 차량 이용률 변화",
    width=1000,  # 전체 그래프 너비
    height=400,  # 전체 그래프 높이
)

# x축과 y축 이름 설정
fig.update_xaxes(title_text="기간", row=1, col=1)
fig.update_xaxes(title_text="기간", row=1, col=2)
fig.update_yaxes(title_text="차량 이용률", row=1, col=1)
fig.update_yaxes(title_text="차량 이용률", row=1, col=2)

# 그래프 출력
fig.show()


### 축제 기간 시간대별 평균 통행량
`분석 목표` : 축제 기간 중 시간대별 통행량 추이 분석

#### 필터링 조건 적용 및 연산

In [107]:
jinju_move = time_moved(jinju, parks) ; incheon_move = time_moved(incheon, ipark) # 축제지 인근 통행량 파악을 위한 데이터 필터링
df_move = pd.concat([jinju_move, incheon_move])

df_move = df_move[df_move['기간'] == '축제'] # 축제기간 데이터만 사용
df_counts = df_move.groupby(['축제', 'date', 'time']).size().reset_index(name='count') # 날짜, 시간대별 통행 횟수 계산
df_avg_counts = df_counts.groupby(['축제', 'time'])['count'].mean().reset_index(name='avg_count') # 시간대별 평균 통행량

df_avg_counts.head(3)

,축제,time,avg_count
0,소래포구 축제,08:00,597.0
1,소래포구 축제,09:00,608.0
2,소래포구 축제,10:00,598.0


In [115]:
df_move['date'] = pd.to_datetime(df_move['date'], format='%Y%m%d') # 날짜 열을 datetime 형식으로 변환
df_move['is_weekend'] = df_move['date'].dt.dayofweek >= 5  # 토요일(5) 또는 일요일(6)일 때 True
df_move['is_weekend'] = df_move['is_weekend'].apply(lambda x: '주말' if x else '평일') # 'is_weekend' 열을 '주말' 또는 '평일'로 표시

df_sep = df_move.groupby(['축제', 'date', 'time', 'is_weekend']).size().reset_index(name='count') # 날짜, 시간대별 통행 횟수 계산
df_avg_sep = df_sep.groupby(['축제', 'time', 'is_weekend'])['count'].mean().reset_index(name='avg_count') # 시간대별 평균 통행량


#### 시각화 결과

In [110]:
import plotly.graph_objects as go
fig = go.Figure()

incheon_move = df_avg_counts[df_avg_counts['축제'] == '소래포구 축제']
jinju_move = df_avg_counts[df_avg_counts['축제'] == '진주남강유등 축제']

# incheon 데이터 선 추가
fig.add_trace(go.Scatter(x=incheon_move['time'], y=incheon_move['avg_count'], mode='lines', 
                         name='소래포구축제', line=dict(color='#0e6d62')))

# jinju 데이터 선 추가
fig.add_trace(go.Scatter(x=jinju_move['time'], y=jinju_move['avg_count'], mode='lines', 
                         name='진주남강유등축제', line=dict(color='red')))

# 레이아웃 설정
fig.update_layout(
    title='축제 기간 시간대별 통행량',
    xaxis_title='시간대',
    yaxis_title='평균 통행량'
)

# 그래프 출력
fig.show()

In [120]:
import plotly.graph_objects as go
fig = go.Figure()

# 진주남강유등 축제의 주말과 평일 데이터
jinju_weekend = df_avg_sep[(df_avg_sep['축제'] == '진주남강유등 축제') & (df_avg_sep['is_weekend'] == '주말')]
jinju_weekday = df_avg_sep[(df_avg_sep['축제'] == '진주남강유등 축제') & (df_avg_sep['is_weekend'] == '평일')]

# 진주남강유등 축제 주말 데이터 선 추가
fig.add_trace(go.Scatter(x=jinju_weekend['time'], y=jinju_weekend['avg_count'], mode='lines',
                         name='진주남강유등축제 - 주말', line=dict(color='red', dash='solid')))

# 진주남강유등 축제 평일 데이터 선 추가
fig.add_trace(go.Scatter(x=jinju_weekday['time'], y=jinju_weekday['avg_count'], mode='lines',
                         name='진주남강유등축제 - 평일', line=dict(color='red', dash='dash')))

# 레이아웃 설정
fig.update_layout(
    title='진주남강유등 축제 기간 시간대별 주말 및 평일 통행량',
    xaxis_title='시간대',
    yaxis_title='평균 통행량',
    legend_title='축제 및 요일'
)

# 그래프 출력
fig.show()


In [122]:
import plotly.graph_objects as go
fig = go.Figure()

# 소래포구 축제의 주말과 평일 데이터
incheon_weekend = df_avg_sep[(df_avg_sep['축제'] == '소래포구 축제') & (df_avg_sep['is_weekend'] == '주말')]
incheon_weekday = df_avg_sep[(df_avg_sep['축제'] == '소래포구 축제') & (df_avg_sep['is_weekend'] == '평일')]


# 소래포구 축제 주말 데이터 선 추가
fig.add_trace(go.Scatter(x=incheon_weekend['time'], y=incheon_weekend['avg_count'], mode='lines',
                         name='소래포구축제 - 주말', line=dict(color='#0e6d62', dash='solid')))

# 소래포구 축제 평일 데이터 선 추가
fig.add_trace(go.Scatter(x=incheon_weekday['time'], y=incheon_weekday['avg_count'], mode='lines',
                         name='소래포구축제 - 평일', line=dict(color='#0e6d62', dash='dash')))

# 레이아웃 설정
fig.update_layout(
    title='소래포구 축제 기간 시간대별 주말 및 평일 통행량',
    xaxis_title='시간대',
    yaxis_title='평균 통행량',
    legend_title='축제 및 요일'
)

# 그래프 출력
fig.show()
